<a href="https://colab.research.google.com/github/thxsxth/RLMimic/blob/master/Learning_a_Variational_State_Space_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
cd 'drive/My Drive/sepsis3-cohort'

In [ ]:
!pip install pyro-ppl

     |████████████████████████████████| 522kB 2.8MB/s 


In [ ]:
# !pip install tensorboardX

In [ ]:
import torch
import numpy as np
import pandas as pd
import datetime as dt
import random
import time
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
import torch.nn.functional as F
import os
import glob
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence,pad_packed_sequence
import torch.nn.init as weight_init
import scipy


In [ ]:
import pyro
import pyro.distributions as dist
import pyro.poutine as poutine
from pyro.distributions import TransformedDistribution
from pyro.distributions.transforms import affine_autoregressive
from pyro.infer import SVI, JitTrace_ELBO, Trace_ELBO, TraceEnum_ELBO, TraceTMC_ELBO, config_enumerate,TraceGraph_ELBO
from pyro.optim import ClippedAdam
from utils import *
device='cuda' if torch.cuda.is_available() else 'cpu'

### Some Data Cleaning

We have pivoted Vitals, Sofa related scores and labs in CSV files

In [ ]:
vitals=pd.read_csv('../Vitals/Vitals.csv',parse_dates=['charttime']) #pivoted vitals
sofa=pd.read_csv('../pivoted_sofa/pivoted_sofa.csv',parse_dates=['endtime','starttime']) #pivoted sofa
labs=pd.read_csv('../pivoted_labs/Pivoted_labs.csv',parse_dates=['charttime'])


In [ ]:
sofa['Vaso']=sofa['rate_norepinephrine'].add(0.1*sofa['rate_dopamine'],fill_value=0).add(sofa['rate_epinephrine'],fill_value=0)
sofa['Vaso']=sofa['Vaso'].add(sofa['rate_dobutamine'],fill_value=0).fillna(0)

In [ ]:
vitals['TempC']=vitals['TempC'].ffill()
sofa['GCS_min']=sofa['GCS_min'].ffill()
labs['icustay_id']=labs['ICUSTAY_ID']


The following, takes the cohort information and re indexes 

In [ ]:
co=pd.read_csv('sepsis3_adults.csv',
               parse_dates=['intime','outtime','suspected_infection_time_poe']) #cohort + demographics
co=co.set_index('icustay_id')

Takes the deathtime from admissions table and include it in the Cohort

In [ ]:
admissions=pd.read_csv('admissions.csv',parse_dates=['ADMITTIME','DISCHTIME','DEATHTIME'])
admissions=admissions.set_index('icustay_id').sort_index()
co['death_time']=admissions['DEATHTIME']

##### Input comes in two seperate Databases, the following combines them together

In [ ]:
input_cv=pd.read_csv('../Fluids/cleaned_input_cv.csv',parse_dates=['charttime']) 
input_mv=pd.read_csv('../Fluids/input_eventsMV.csv',parse_dates=['starttime','endtime'])

In [ ]:
input_cv=input_cv[['icustay_id','charttime','tev']]
input_mv=input_mv[['icustay_id','endtime','tev']]
input_mv['tev_mv']=input_mv['tev']
input_mv['charttime']=input_mv['endtime']
input_mv=input_mv.drop('tev',axis=1)
input_fluids=input_mv.merge(input_cv,on=['icustay_id','charttime'],how='outer')[['icustay_id','charttime','tev','tev_mv']]

In [ ]:
input_fluids['volume']=input_fluids['tev'].add(input_fluids['tev_mv'],fill_value=0)

### Normalizing all the data
Means were precomputed, but can be easily derived from df.describe() or df.loc[].mean()

In [ ]:
input_fluids['volume']=(input_fluids['volume']-7.508081e+01)/5.910144e+02
input_fluids=input_fluids.drop(['tev','tev_mv'],axis=1)

In [ ]:
vital_means=pd.Series([1.014094e+02,1.215081e+02, 6.027904e+01 ,7.915051e+01 ,2.020732e+01,3.689988e+01,9.708047e+01],
                      index=['HeartRate','SysBP','DiasBP'	,'MeanBP'	,'RespRate'	,'TempC','SpO2'])

std_vitals=pd.Series([3.279276e+01,2.368822e+01,1.455537e+01,1.673107e+01,6.043442e+00,7.760427e-01,3.495480e+00],index=['HeartRate','SysBP','DiasBP'	,'MeanBP'	,'RespRate'	,'TempC','SpO2'])

In [ ]:
##Normalize Vitals
for vital in vital_means.index:
   vitals[vital]=(vitals[vital]-vital_means[vital])/std_vitals[vital]

In [ ]:
sofa.describe()

,icustay_id,hr,PaO2FiO2Ratio_novent,PaO2FiO2Ratio_vent,rate_epinephrine,rate_norepinephrine,rate_dopamine,rate_dobutamine,MeanBP_min,GCS_min,urineoutput,bilirubin_max,creatinine_max,platelet_min,respiration,coagulation,liver,cardiovascular,cns,renal,respiration_24hours,coagulation_24hours,liver_24hours,cardiovascular_24hours,cns_24hours,renal_24hours,SOFA_24hours,Vaso
count,5.380769e+06,5.380769e+06,9174.000000,286131.000000,43579.000000,335605.000000,91099.000000,34683.000000,4.615092e+06,5.380768e+06,3.178315e+06,77091.000000,333873.000000,314318.00000,295305.000000,314318.000000,77091.00000,4.636356e+06,1.508665e+06,4.732542e+06,5.380769e+06,5.380769e+06,5.380769e+06,5.380769e+06,5.380769e+06,5.380769e+06,5.380769e+06,5.380769e+06
mean,2.498804e+05,1.580930e+02,252.089197,249.689558,0.051119,0.177287,6.320698,4.878103,7.817685e+01,1.177083e+01,1.687066e+02,4.012415,1.545177,217.11330,1.173150,0.596005,1.13021,5.897353e-01,1.467729e+00,4.285035e-01,6.920319e-01,4.702023e-01,2.968944e-01,1.055728e+00,1.805929e+00,1.282797e+00,5.603582e+00,5.361584e-02
std,2.878473e+04,2.377073e+02,145.262215,115.511308,0.218740,0.493985,6.742377,3.595475,1.602817e+01,3.691253e+00,2.621426e+03,7.413100,1.550674,141.69615,1.506095,0.928538,1.35431,1.018574e+00,1.368434e+00,1.135888e+00,1.294751e+00,8.540253e-01,8.471924e-01,1.034393e+00,1.453541e+00,1.532017e+00,3.798351e+00,5.292760e-01
min,2.000010e+05,0.000000e+00,19.000000,2.500000,0.000700,0.000096,0.080000,0.100000,2.000000e-01,3.000000e+00,-1.440000e+04,0.100000,0.050000,5.00000,0.000000,0.000000,0.00000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.248890e+05,2.600000e+01,138.571429,168.333327,0.020000,0.056140,3.000000,2.500000,6.700000e+01,1.000000e+01,4.500000e+01,0.500000,0.700000,119.00000,0.000000,0.000000,0.00000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00
50%,2.499450e+05,7.000000e+01,228.000000,232.000000,0.030000,0.101587,5.000000,4.006904,7.600000e+01,1.400000e+01,1.000000e+02,1.100000,1.000000,189.00000,0.000000,0.000000,0.00000,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,2.000000e+00,1.000000e+00,5.000000e+00,0.000000e+00
75%,2.747130e+05,1.930000e+02,346.000000,312.000000,0.050000,0.201159,8.000000,5.046614,8.766670e+01,1.500000e+01,1.900000e+02,3.800000,1.700000,279.00000,3.000000,1.000000,2.00000,1.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,3.000000e+00,3.000000e+00,8.000000e+00,0.000000e+00
max,2.999990e+05,4.154000e+03,2433.333333,1929.166710,19.244444,61.439027,523.784242,50.000000,2.990000e+02,1.500000e+01,4.555555e+06,82.800000,138.000000,2292.00000,4.000000,4.000000,4.00000,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,2.400000e+01,6.143903e+01


In [ ]:
sofa_means=pd.Series([5.361584e-02, 2.968944e-01,1.055728e+00,1.805929e+00,1.282797e+00,5.603582e+00],
                      index=['Vaso','liver_24hours'	,'cardiovascular_24hours','cns_24hours'	,'renal_24hours',	'SOFA_24hours'])

sofa_stds=pd.Series([5.292760e-01, 8.471924e-01,1.034393e+00,1.453541e+00,1.532017e+00,3.798351e+00],index=['Vaso','liver_24hours','cardiovascular_24hours','cns_24hours'	,'renal_24hours',	'SOFA_24hours'])

In [ ]:
#  sofa_means=pd.Series([0.619876, 2.968944e-01,1.055728e+00,1.805929e+00,1.282797e+00,5.603582e+00],
#                       index=['Vaso','liver_24hours'	,'cardiovascular_24hours','cns_24hours'	,'renal_24hours',	'SOFA_24hours'])

# sofa_stds=pd.Series([1.699333, 8.471924e-01,1.034393e+00,1.453541e+00,1.532017e+00,3.798351e+00],index=['Vaso','liver_24hours','cardiovascular_24hours','cns_24hours'	,'renal_24hours',	'SOFA_24hours'])

In [ ]:
for sofa_ in sofa_means.index:
   sofa[sofa_]=(sofa[sofa_]-sofa_means[sofa_])/sofa_stds[sofa_]

In [ ]:
# sofa['Vaso']=sofa['Vaso'].fillna(0)

In [ ]:
co['age']=(co['age']-66.052869)/16.179046	
co['Weight']=(co['Weight']-81.135925)/24.059419	

### Dataset and Dataloader
We subclass PyTorch Dataset class to access patient level trajectories. Then we use a customized collate function to batch into minibatches after padding

In [ ]:
def get_mini_batch_mask(mini_batch, seq_lengths):
    mask = torch.zeros(mini_batch.shape[0:2])
    for b in range(mini_batch.shape[0]):
        mask[b, 0:seq_lengths[b]] = torch.ones(seq_lengths[b])
    return mask.to(device)

In [ ]:
class modeling_dataset(Dataset):
  """
  Implements a dataset for patients
  Needs Vitals,Sofa,Inputs,co tables

  This is only used for Model Learning, Rewards, Terminals are unnecessary
  """
  def __init__(self,patient_ids,train=True):
    #patient_ids :List/np.array
    self.ids=patient_ids
    self.train=train

  def __len__(self):
    return len(self.ids)

  def __getitem__(self,idx):
    #Get Patient from the index
    pat=self.ids[idx]
    pat_fluids=input_fluids[input_fluids.icustay_id==pat].set_index('charttime')
    pat_sofa=sofa[sofa.icustay_id==pat].set_index('endtime')
    pat_sofa=pd.concat([pat_sofa,pat_fluids]).resample('H').sum()

    pat_vitals=vitals[vitals.icustay_id==pat].set_index('charttime')
    pat_labs=labs[labs.icustay_id==pat]
    pat_df=pd.concat([pat_vitals,
                              pat_sofa]).resample('H').last()[['HeartRate','SysBP','DiasBP',	'MeanBP','RespRate','SpO2','TempC',
                                      'liver_24hours','cardiovascular_24hours',
                                      'cns_24hours','renal_24hours','SOFA_24hours','volume','Vaso']].resample('H').last()
    # pat_df=pd.concat([pat_vitals,
    #                           pat_sofa]).resample('H').last()[['HeartRate','SysBP','DiasBP',	'MeanBP','RespRate','SpO2',
    #                                   'volume','Vaso']].resample('H').last()
  
    """
    TO DO: 
    Implement get_rewards
    Get age gender and if they died may be weight height
    GET TREATMENTS: Make it Tensor DONE
    GET Trajectory Make it Tensor DONE
    """
    dead=co.loc[pat].HOSPITAL_EXPIRE_FLAG==1
    if co.loc[pat].HOSPITAL_EXPIRE_FLAG==1:
          pat_df=pat_df.truncate(after=co.loc[pat].death_time)

    pat_df=pat_df.ffill().dropna()
    # print(pat_df.shape)

    constants=torch.FloatTensor(co.loc[pat][['age','is_male','Weight']]).to(device)

    if pat_df.shape[0]>47:
          k=np.random.choice(pat_df.shape[0]-47)
          pat_df=pat_df.iloc[k:k+47,:]

    
    treatments=torch.FloatTensor(pat_df[['Vaso','volume']].values).to(device)     
    trajectory=torch.FloatTensor(pat_df.drop(['Vaso','volume'],axis=1).values).to(device)

    return trajectory,treatments, constants
    


In [ ]:
def collate_model(batch_data):

  """
  We will be a list of tuples,
  len(list) will be batch_size
  (trajectory,treatments, constants,dead) for each patient in batch
  """
  trajectories=[]
  treatments=[]
  seq_lens=[]
  constants_list=[]
  dead_=[]

  for (trajectory,treatment,constants) in batch_data:

    trajectories.append(trajectory) 
    treatments.append(treatment)
    seq_lens.append(trajectory.shape[0])
    constants_list.append(constants.unsqueeze(0))
    # dead_.append(dead)

  padded_trajectories=pad_sequence(trajectories,batch_first=True)
  padded_treatments=pad_sequence(treatments,batch_first=True)
  constants=torch.cat(constants_list)
  mask=get_mini_batch_mask(padded_trajectories,seq_lens)

  return padded_trajectories,padded_treatments,constants,mask,seq_lens

#### The Training and Validation Cohorts. The Modelling Cohort was pre selected using previous work

In [ ]:
patient_list=list(np.load('new_pats.npy'))

In [ ]:
training_cohort=np.random.choice(a=patient_list, size=int(0.8*len(patient_list)), replace=False, p=None)
train_dataset=modeling_dataset(training_cohort)
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True,collate_fn=collate_model)

In [ ]:
valid_cohort=list(set(patient_list)-set(training_cohort))
valid_dataset=modeling_dataset(valid_cohort)
valid_loader=DataLoader(valid_dataset,batch_size=16,shuffle=False,collate_fn=collate_model)

### Defining Models and hyperparamters

In [ ]:
class patient_encoder(nn.Module):

  """
  Takes constant demographic details and possibly labs and encodes into  z_0, h_0
  """

  def __init__(self,constant_dim,z_dim,h_dim,hidden_dim=64):
    
    super(patient_encoder,self).__init__()
    self.linear_input=nn.Linear(constant_dim,hidden_dim)
    self.h0=nn.Linear(hidden_dim,h_dim)
    self.z_mu=nn.Linear(hidden_dim,z_dim)
    self.z_sigma=nn.Linear(hidden_dim,z_dim)
    
    

  def forward(self,x):
    """
    X has shape batch_size*input_dim or num_particles*batch_size*input_dim
    """
    x=F.elu(self.linear_input(x)) #batch_size*hidden_dim
    h0=self.h0(x)
    s0=self.z_mu(x)
    # z_sigma=F.softplus(self.z_sigma(x))

    return h0,s0

In [ ]:
class Encoder(nn.Module):
  """ Encodes x_{:t} for the variational distribution
     adopted  from https://github.com/guxd/deepHMM/blob/master/modules.py#L163
  """

  def __init__(self,obs_dim,h_dim,a_dim,n_layers=1, dropout=0.0, noise_radius=0.2):
      
      super(Encoder,self).__init__()
      
      self.rnn=nn.GRU(obs_dim,h_dim,n_layers,batch_first=True)
      self.dropout=dropout
      
      self.noise_radius=noise_radius
      
      self.n_layers=n_layers
      self.hidden_dim=h_dim
      
      # self.init_h = nn.Parameter(torch.randn(n_layers,1,
      #                                        hidden_dim), requires_grad=True)
      self.init_weights()

  def  init_weights(self):
        for w in self.rnn.parameters(): # initialize the gate weights with orthogonal
            if w.dim()>1:
                weight_init.orthogonal_(w)

  def forward(self,obs,obs_lens,patient_demos,init_h=None, noise=False):

    """
    obs: A mini batch of observations B*T*D
    obs_lens=observation lengths to pack pad sequences
    patient_demos: Batch of size B*pat_dim (=C)
    """
    batch_size, max_len, freq=obs.size()

    # obs=F.dropout(obs,training=self.training)  #B*T*D
    
    obs_lens=torch.LongTensor(obs_lens).to(device)
    obs_lens_sorted, indices = obs_lens.sort(descending=True)
    obs_sorted = obs.index_select(0, indices)  
    
    packed_obs=pack_padded_sequence(obs_sorted,obs_lens_sorted.data.tolist(),batch_first=True)

    if init_h is None:
        init_h,_=self.patient_encoder(patient_demos).unsqueeze(0)

    init_h=init_h.unsqueeze(0) #1*B*H
  
    hids, h_n = self.rnn(packed_obs, init_h) # hids: [B x T x H]  
                                                  # h_n: [num_layers*B*H)
    _, inv_indices = indices.sort()

    hids, lens = pad_packed_sequence(hids, batch_first=True)         
    hids = hids.index_select(0, inv_indices) 
    
      #[B*T*H]
    # h_n = h_n.index_select(1, inv_indices)     #[1*T*H]
    

    # h_n = h_n.view(1, batch_size, self.hidden_dim) #[n_layers x n_dirs x batch_sz x hid_sz]
    
    # h_n = h_n[-1] # get the last layer [1 x B x H]
    # enc = h_n.transpose(0,1).contiguous().view(batch_size,-1) #[B*H]
     
    # if noise and self.noise_radius > 0:
    #      gauss_noise = torch.normal(means=torch.zeros(enc.size(), device=inputs.device),std=self.noise_radius)
    #      enc = enc + gauss_noise
            
    return hids


In [ ]:
class Proposer(nn.Module):
    """
    Parameterizes `q(z_t | a_{t-1}, o_{:t}, a_{t})` the proposal distribution
    """

    def __init__(self, z_dim, h_dim,a_dim,constant_dim,n_layers=2,mu_layers=1,sigma_layers=1,hidden_dim=64):
        
        super().__init__()
        self.z_dim=z_dim
        input_size=z_dim+h_dim+a_dim+constant_dim
        self.bn=nn.BatchNorm1d(input_size)
        self.linear_input=nn.Linear(input_size,hidden_dim)
        self.hiddens=nn.ModuleList([nn.Sequential(nn.Linear(hidden_dim,hidden_dim),nn.ELU(),nn.BatchNorm1d(hidden_dim)) for j in range((mu_layers))])

        self.hiddens_mu=nn.ModuleList([nn.Sequential(nn.Linear(hidden_dim,hidden_dim),nn.ELU(),nn.BatchNorm1d(hidden_dim)) for j in range((mu_layers))])
        self.hiddens_sigma=nn.ModuleList([nn.Sequential(nn.Linear(hidden_dim,hidden_dim),nn.ELU()) for j in range((sigma_layers))])
        
        
        self.mu_linear=nn.Linear(hidden_dim,z_dim)
        self.sigma_linear=nn.Linear(hidden_dim,z_dim)
        self.sigma_param=nn.Parameter(torch.ones(z_dim).to(device)*0.001)


    def forward(self, z_t_1, h_t,a_t,constants):
      """
      inputs have batch_size*dim_ or alternatively batch_size*K*dim_
      where k is the number of particles for SMC

      """        batch_size=z_t_1.shape[0]
        
        input_=torch.cat([z_t_1,h_t,a_t,constants],dim=1)   #Batch_size*(z_dim_h_dim+a_time)=S
        
        
        input_=self.bn(input_) #B*S
       
    
        
    
        input_=self.linear_input(input_)
        # print(input_.shape)  #B*K*H

        for layer in self.hiddens:
            input_=layer(input_)  #B*K*H

        mu=input_
        sigma=input_

        for layer in self.hiddens_mu:
          mu=layer(mu)  #B*K*H

        mu=self.mu_linear(mu) #B*K*Z

        for layer in self.hiddens_sigma:
           sigma=self.sigma_linear(sigma) #B*K*Z

        sigma=F.softplus(sigma)

      
        # sigma = self.sigma_param.expand(batch_size,self.z_dim) #B*Z_dim



        return mu, sigma

In [ ]:
class Emitter(nn.Module):
  """
  A MLP representing  p(o_t|z_t)
  """
  def __init__(self,z_dim,continous_dim,poison_dim=None,hidden_size=128,hidden_layers=2):
    
    super(Emitter,self).__init__()
    
    self.cont_dim=continous_dim
    self.bn=nn.BatchNorm1d(z_dim)
    self.linear=nn.Linear(z_dim,hidden_size)
    self.hidden_layers=nn.ModuleList([nn.Sequential(nn.Linear(hidden_size,hidden_size),nn.ELU(),nn.BatchNorm1d(hidden_size)) for i in range(hidden_layers)])

    self.linear_mu=nn.Linear(hidden_size,continous_dim)
    self.linear_sigma=nn.Linear(hidden_size,continous_dim)
    self.poison_dim=poison_dim
    
    if poison_dim:
      self.linear_poison=nn.Linear(hidden_size,poison_dim)

    self.sigma_param=nn.Parameter(torch.ones(self.cont_dim).to(device)*0.1)

  def forward(self,z_t):
    """
    z has dimensions #batch_size*num_particles*Z_dim
    """
    # z_t=z_t.permute(0,2,1)
    # z=self.linear(self.bn(z_t))
    batch_size=z_t.shape[0]
    z=self.linear(z_t)
   
    for layer in self.hidden_layers:
          z=layer(z)
    
    mu=self.linear_mu(z)
    sigma=F.softplus(self.linear_sigma(z))
    if self.poison_dim:
      poison=F(self.linear_poison(z))
      return mu, sigma,poison

    sigma = self.sigma_param.expand(batch_size,self.cont_dim) #B*Z_dim

    return mu,sigma  #Batch_size*K*Obs_dim

class Gated_Transition(nn.Module):
    """
    Parameterizes the gaussian latent transition probability p(z_t | z_{t-1},a_{t-1},h_{t-1})
    using a MLP, where z is latent and a is the action
    Modified from https://pyro.ai/examples/dmm.html
    """
    def __init__(self, z_dim,a_dim,h_dim,constant_dim, hidden_dim,n_layers=3):
      super(Gated_Transition,self).__init__()
      
      input_dim=z_dim+a_dim+h_dim+constant_dim

      self.bn=nn.BatchNorm1d(input_dim)
      self.input_to_h=nn.Linear(input_dim, hidden_dim)
      self.hidden_layers=nn.ModuleList([nn.Sequential(nn.Linear(hidden_dim,hidden_dim),nn.ELU(),nn.BatchNorm1d(hidden_dim)) for i in range(n_layers)])
      self.hidden_to_z=nn.Linear(hidden_dim,z_dim)
      
      self.proposed_mean_to_h=nn.Linear(input_dim, hidden_dim)
      self.proposed_h_to_z=nn.Linear(hidden_dim,z_dim)

      self.input_to_mean=nn.Linear(input_dim,z_dim)
      self.input_to_scale=nn.Linear(input_dim,z_dim)
      self.input_to_hid_scale=nn.Linear(input_dim,hidden_dim)
      self.hidden_to_scale=nn.Linear(hidden_dim,z_dim)

      self.input_to_mean.bias.data = torch.zeros(z_dim)


    def forward(self,z_prev,a_t,h_t,constants):
  
      """
        Given the latent z_{t-1} and the action u_{t-1}
        
        we return the mean and scale vectors that parameterize the
        (diagonal) gaussian distribution p(z_t | z_{t-1},a_{t-1},h_{t-1})
        
        z_t_1 has shape B*num_particles*Z_dim or B*Z_dim
        a_t_1 has shape B*num_particles*a_dim or B*a_dim
      """
      # concatenate the latent z and actions along the frequency dimension
      input_=torch.cat([z_prev,a_t,h_t,constants],dim=1)  #B*K*(D+T) let's call D+T=F
      
      # input_=self.bn(input_)

      #calculate the gate
      # Add hidden layers if necessary
      
      _gate=F.elu(self.input_to_h(input_))
      gate=F.sigmoid(self.hidden_to_z(_gate))
      

      # compute the 'proposed mean'
    
      _proposed_mean = F.elu(self.proposed_mean_to_h(input_))  #B*K*U_D-->B*K*H_D   
      for layer in self.hidden_layers:
          _proposed_mean=layer(_proposed_mean)   #B*K*H_D
    
      proposed_mean = self.proposed_h_to_z(_proposed_mean)  #B*K*H_D--->B*K*Z_D
      
      
      
      mean = (1 - gate) * self.input_to_mean(input_) + gate * proposed_mean #B*K*Z_D
      # mean = gate * proposed_mean #B*Z_D
      
      _scale=F.elu(self.input_to_hid_scale(input_)) #B*K*F-->B*K*H
    
      for layer in self.hidden_layers:
        _scale=layer(_scale)     #B*K*H
      
      scale=F.softplus(self.hidden_to_scale(_scale))   #B*K*Z_D

      
                        
      return mean,scale

In [ ]:
class DMM(nn.Module):
  
  """
    This PyTorch Module encapsulates the model as well as the
    variational distribution (the guide) for the Deep Markov Model

    Modified from https://github.com/pyro-ppl/pyro/blob/dev/examples/dmm/dmm.py
  """
  def __init__(self,z_dim,a_dim,obs_dim,constants_dim,h_dim,binary_dim=None,
               hidden_emitter_dim=512,hidden_gated_dim=512,
               ,num_iafs=0, iaf_dim=50):
    
    """
    z_dim (int) : Dimension of the Latent Space
    a_dim (int) : Dimension of Action space
    obs_dim (int) : Dimension of (conitnous) observations

    binary_dim (int) : Dimension of binary observations
    hidden_layers (iterable): Number of hidden layers for Emitter, transm Encoder respectively
    Others should be self explanatory
    """
    
    super(DMM,self).__init__()
    self.initial=patient_encoder(constants_dim,z_dim,h_dim)
    self.emitter=Emitter(z_dim,obs_dim,binary_dim,hidden_emitter_dim)
    self.trans=Gated_Transition(z_dim,a_dim,h_dim,constants_dim, hidden_dim=64)
    self.proposer=Proposer(z_dim,h_dim,a_dim,constants_dim,n_layers=5,mu_layers=2,sigma_layers=1,hidden_dim=64)
    self.rnn=Encoder(obs_dim,h_dim,a_dim,n_layers=1)
    
   
    input_dim=z_dim+a_dim 
    if binary_dim:
      self.rnn=Encoder(obs_dim+binary_dim,hidden_dim=h_dim,n_layers=hidden_layers[2])
    else:
      self.rnn=Encoder(obs_dim,h_dim,a_dim)
    
    # if we're using normalizing flows, instantiate those 
    self.iafs = [affine_autoregressive(z_dim, hidden_dims=[iaf_dim]) for _ in range(num_iafs)]
    self.iafs_modules = nn.ModuleList(self.iafs)

    if device=='cuda':
      self.cuda()



    def model(self,observations,obs_lens,actions,mask,constants,binary=None,
               annealing_factor=1.0):
      """
      batch : Batch of continous observables: B*T*O
      binary: Batch of binary observables:
      batch_lengths :list
      actions : B*T*|A|
      Constants B*C
      """
      
      T_max = observations.size(1)

      # set z_prev = z_0 to setup the recursive conditioning in p(z_t | z_{t-1,s_{t-1}})
      # and set initial treatment to zero
      
      z_prev,h_prev=self.initial(constants)  #B*Z_dim, B*h_dim
      h_prev=h_prev.contiguous()

      a_prev=torch.zeros(actions.shape[0],actions.shape[2]).to(device) #B*A
      rnn_output=self.rnn(observations,obs_lens,constants,h_prev)  #rnn_ouput has shape B*T*H
      
                 
      # we enclose all the sample statements in the model in a plate.
      # this marks that each datapoint is conditionally independent of the others
      
      with pyro.plate("z_minibatch", len(observations)):

        for t in pyro.markov(range(1, T_max + 1)):

          # the next chunk of code samples z_t ~ p(z_t | z_{t-1},u_{t-1})
          # note that (both here and elsewhere) we use poutine.scale to take care of KL annealing

          z_mean,z_scale=self.trans(z_prev,a_prev,rnn_output[:,t-1,:],constants)

          with poutine.scale(scale=annealing_factor):              
                    z_t = pyro.sample("z_%d" % t,
                                      dist.Normal(z_mean, z_scale)
                                          .mask(mask[:, t - 1:t])
                                          .to_event(1))
          
          if binary:
            mu,sigma,binary=self.emitter(z_t)
          else:            
            mu, sigma=self.emitter(z_t)

          pyro.sample("cts_x_%d" % t,
                            dist.Normal(mu,sigma)
                                .mask(mask[:, t - 1:t])
                                .to_event(1),
                            obs=observations[:, t - 1, :])
          
          if binary:        
               pyro.sample("binary_x_%d" % t,
                            dist.Bernoulli(binary)
                                .mask(mask[:, t - 1:t])
                                .to_event(1),
                            obs=binary[:, t - 1, :])
          z_prev = z_t
          a_prev=actions[:,t-1,:]
    
    def guide(self,observations,obs_lens,actions,mask,constants,binary=None,
               annealing_factor=1.0):

      # this is the number of time steps we need to process in the mini-batch
      T_max = observations.size(1)
      # register all PyTorch (sub)modules with pyro
      pyro.module("dmm", self)

       # if on gpu we need the fully broadcast view of the rnn initial state
       # to be in contiguous gpu memory
       
      z_prev,h_prev=self.initial(constants)  #B*Z_dim, B*h_dim
      h_prev=h_prev.contiguous()
      rnn_output=self.rnn(observations,obs_lens,constants,h_prev)  #rnn_ouput has shape B*T*H
       
      a_prev=torch.zeros(actions.shape[0],actions.shape[2]).to(device)

      with pyro.plate("z_minibatch", len(observations)):
            
            # sample the latents z one time step at a time
            # we wrap this loop in pyro.markov so that TraceEnum_ELBO can use multiple samples from the guide at each z
            
            for t in pyro.markov(range(1, T_max + 1)):
                
                z_loc, z_scale = self.proposer(z_prev, rnn_output[:, t - 1, :],a_prev,constants)

                # if we are using normalizing flows, we apply the sequence of transformations
                # parameterized by self.iafs to the base distribution defined in the previous line
                # to yield a transformed distribution that we use for q(z_t|...)
                
                if len(self.iafs) > 0:
                    z_dist = TransformedDistribution(dist.Normal(z_loc, z_scale), self.iafs)
                    assert z_dist.event_shape == (self.z_q_0.size(0),)
                    assert z_dist.batch_shape[-1:] == (len(observations),)
                
                else:
                    z_dist = dist.Normal(z_loc, z_scale)
                    assert z_dist.event_shape == ()
                    # assert z_dist.batch_shape[-2:] == (len(observations), z_prev.size(0))

                # sample z_t from the distribution z_dist
               
                with pyro.poutine.scale(scale=annealing_factor):
                    if len(self.iafs) > 0:
                        # in output of normalizing flow, all dimensions are correlated (event shape is not empty)
                        z_t = pyro.sample("z_%d" % t,
                                          z_dist.mask(mask[:, t - 1]))
                    
                    else:
                        # when no normalizing flow used, ".to_event(1)" indicates latent dimensions are independent
                        z_t = pyro.sample("z_%d" % t,
                                          z_dist.mask(mask[:, t - 1:t])
                                          .to_event(1))
                
                
                z_prev = z_t
                u_prev=actions[:,t-1,:]

    self.model=model
    self.guide=guide


def get_annhealing_factor(epoch):
  if epoch<10:
    return 0.25
  else:
     return min(1.0,0.25+0.05*epoch)



In [ ]:

def validate(trajectories,treatments,constants,mask,obs_lens):
    # put the  into evaluation mode (i.e. turn off drop-out if applicable)
    dmm.rnn.eval()
    dmm.emitter.eval()
    dmm.trans.eval()

    val_nll = svi.evaluate_loss(dmm,trajectories,obs_lens,treatments,mask,constants,
               binary=None,annealing_factor=annhealing_factor
                               ) / np.sum(obs_lens)

    dmm.rnn.train()
    dmm.emitter.train()
    dmm.trans.train()
    return val_nll

In [47]:

"""### Defining Model and hyperparamters"""

learning_rate=0.001
beta1=0.96
beta2=0.999
clip_norm=5.0
lr_decay=0.95
weight_decay=0.0


dmm=DMM(64,2,12,3,64)

N_epochs=5000
annhealing_factor=1.0

adam_params = {"lr": learning_rate, "betas": (beta1, beta2),
                   "clip_norm": clip_norm, "lrd": lr_decay,
                   "weight_decay": weight_decay}
optimizer = ClippedAdam(adam_params)

# setup inference algorithm
svi = SVI(dmm.model, dmm.guide, optimizer, Trace_ELBO(num_particles=5))


In [ ]:
# dmm.load_state_dict(torch.load('state_dict_val_best.pt', map_location=torch.device('cpu')))

<All keys matched successfully>

### Training

In [ ]:
times = [time.time()]
for epoch in range(3,N_epochs):
  val_nll=0
  train_nll=0
  val_steps=0
  train_steps=0
  dmm.train()
  # for i,(batch,masks,actions,batch_lens) in enumerate(train_loader):
  for i,(trajectories,treatments,constants,mask,obs_lens) in enumerate(train_loader):

    # obs_lens=[24 for i in range(trajectories.shape[0])]

    if min(obs_lens)<2:
      continue


    loss = svi.step(dmm,trajectories,obs_lens,treatments,mask,constants,
               binary=None,annealing_factor=annhealing_factor)
    
    print(loss)
    
    batch_nll=svi.evaluate_loss(dmm,trajectories,obs_lens,treatments,mask,constants
                                 ) / np.sum(obs_lens)

    # if batch_nll<100:  
    #    torch.save(dmm.state_dict(),'state_dict_49.pt')
    
    
    train_nll+=batch_nll
    train_steps+=1
    print(batch_nll)
    print('Epoch ',epoch,'Batch : ', train_steps, ' Training Loss :',train_nll/train_steps,end='')

   
    
    
     
  val_nll=0
  val_steps=0
  for trajectories,treatments,constants,mask,obs_lens in valid_loader:
      if min(obs_lens)==0:
          continue
      
      val_nll+=validate(trajectories,treatments,constants,mask,obs_lens)
      print('Validating : ',validate(trajectories,treatments,constants,mask,obs_lens))
      val_steps+=1 

      
      print(val_nll/val_steps) 
  
  if val_nll/val_steps<40:
          torch.save(dmm.state_dict(),'state_dict_{}.pt'.format(val_nll/val_steps))
       
     


  print('-'*125)
  if val_nll/val_steps<30:  
    torch.save(dmm.state_dict(),'state_dict_{}.pt'.format(val_nll/val_steps))
  
  # writer.add_scalar('train_nll', train_nll/train_steps, epoch)
  print('Train nll {}'.format(train_nll/train_steps))
  # writer.add_scalar('vall_nll',val_nll/val_steps, epoch)
  print('Validation nll {}'.format(val_nll/val_steps))  
  # annhealing_factor=get_annhealing_factor(epoch)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


32243.592254638672
23.134495449678802
Epoch  3 Batch :  1  Training Loss : 23.13449544967880225045.700213623048
17.349431426602344
Epoch  3 Batch :  2  Training Loss : 20.24196343814057329357.507577514643
20.921051217765044
Epoch  3 Batch :  3  Training Loss : 20.46832603134872725996.42530517578
18.76891124636628
Epoch  3 Batch :  4  Training Loss : 20.04347233510311830471.713769531252
21.13378731021394
Epoch  3 Batch :  5  Training Loss : 20.2615353301252828738.426162719727
19.910382543103452
Epoch  3 Batch :  6  Training Loss : 20.2030098656216422772.161486816407
16.101307038407327
Epoch  3 Batch :  7  Training Loss : 19.6170523188767428211.18526306152
20.707481268274854
Epoch  3 Batch :  8  Training Loss : 19.75335593755150431017.821862792967
21.328763615145228
Epoch  3 Batch :  9  Training Loss : 19.92840123506191623323.899560546877
16.799601426235466
Epoch  3 Batch :  10  Training Loss : 19.6155212541792726727.874435424805
18.586425507703083
Epoch  3 Batch :  11  Training Loss : 1